### Scraping of Titles

In [40]:
import praw
import numpy as np
import pandas as pd
from praw.models import MoreComments

reddit_read_only = praw.Reddit(client_id="kolb_847uU2R6NfDKrjcSg",         # your client id
                               client_secret="42dvojR3nEVVMQNG-4KZ5A0_4zPOSg",      # your client secret
                               user_agent="Unstructured_Data_Final_Project by /u/Single-Cookie-194")        # your user agent

In [36]:
subreddit = reddit_read_only.subreddit("dating_advice")

posts = subreddit.top(time_filter='year', limit = 5000)
# Scraping the top posts of the current month
 
postsDict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": []
              }
 
for post in posts:
    # Title of each post
    postsDict["Title"].append(post.title)
     
    # Text inside a post
    postsDict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
    postsDict["ID"].append(post.id)
     
    # The score of a post
    postsDict["Score"].append(post.score)
     
    # Total number of comments inside the post
    postsDict["Total Comments"].append(post.num_comments)
     
    # URL of each post
    postsDict["Post URL"].append(post.url)
 
# Saving the data in a pandas dataframe
topPosts = pd.DataFrame(postsDict)
topPosts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...
1,My father told me this,\n\nMy father told me this when he walked up...,sseson,8905,256,https://www.reddit.com/r/dating_advice/comment...
2,He said yes! I'm finally got the first date of...,"UPDATE: Oh my god, I can't believe how many co...",sdvk7s,8482,504,https://www.reddit.com/r/dating_advice/comment...
3,(22 F) Why does every guy my age want a threes...,If one more guy asks me I might snap. A guy I’...,rig0uy,7775,3014,https://www.reddit.com/r/dating_advice/comment...
4,She cancelled our date because of my zodiac sign,"I’m a bit thrown off by this, but I was cancel...",s7g0gm,7155,786,https://www.reddit.com/r/dating_advice/comment...
...,...,...,...,...,...,...
994,He thanked me mid makeout for making out with ...,I’m 29F and I was with 28M. We’ve been talking...,xjio4c,263,154,https://www.reddit.com/r/dating_advice/comment...
995,Men of Reddit... Would you date a teacher? (se...,I'm a teacher and I usually have a tough time ...,vcefib,263,378,https://www.reddit.com/r/dating_advice/comment...
996,How can I attract decent men and not just ones...,To keep this straight I mainly meet men in dat...,ut3zwi,260,389,https://www.reddit.com/r/dating_advice/comment...
997,another guy who doesn't listen,"So, I matched with a guy last week and we got ...",ubny4z,259,105,https://www.reddit.com/r/dating_advice/comment...


In [37]:
# Limiting to top 100 title
top100 = topPosts.head(100)

### Scraping of Comments

In [42]:
# Dictionary with title id and corresponding URL
urlDict = {top100["ID"][i]: top100["Post URL"][i] for i in range(len(top100["ID"]))}

# Formatting for comment dictionary
commentDict = {"Post ID": [], "User": [], "Body": [], "Score": []}

# Pulling comments from each title
for id in urlDict:
    submission = reddit_read_only.submission(id)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        commentDict["Post ID"].append(id)
        commentDict["User"].append(comment.author)
        commentDict["Body"].append(comment.body)
        commentDict["Score"].append(comment.score)

allComments = pd.DataFrame(commentDict)
allComments

,Post ID,User,Body,Score
0,sgcach,AutoModerator,Welcome to /r/dating_advice!\n\nPlease keep th...,1
1,sgcach,fake-august,No one is busier than someone who is not inter...,3037
2,sgcach,Jammer250,"Dating should be effortless, yes, but a succes...",777
3,sgcach,onrappel,"Also you won’t be wondering why, on a weekend ...",1648
4,sgcach,uniqueruntimeerror,"Yeah, and even then they can turn 180 and pret...",144
...,...,...,...,...
55187,sf768x,Fluffy_Telephone4258,Agreed!,2
55188,sf768x,Anti-ThisBot-IB,https://i.imgur.com/KrwA19h.jpeg\n***\n ^(I am...,1
55189,sf768x,Anti-ThisBot-IB,https://i.imgur.com/KrwA19h.jpeg\n***\n ^(I am...,1
55190,sf768x,None,This.,-9


### Combining Titles & Comments

In [49]:
# Rename Post ID column to match name in topPosts & score column
allComments = allComments.rename(columns= {"Post ID": "ID", "Score": "Comment Score"})

# Rename score column in topPosts
top100 = top100.rename(columns= {"Score": "Title Score"})

# Combining topPosts and allComments dataframes
combinedScrape = pd.merge(top100, allComments, on= "ID")

In [50]:
combinedScrape

,Title,Post Text,ID,Title Score,Total Comments,Post URL,User,Body,Comment Score
0,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...,AutoModerator,Welcome to /r/dating_advice!\n\nPlease keep th...,1
1,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...,fake-august,No one is busier than someone who is not inter...,3037
2,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...,Jammer250,"Dating should be effortless, yes, but a succes...",777
3,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...,onrappel,"Also you won’t be wondering why, on a weekend ...",1648
4,"When someone really likes you, it’s effortless.",Just keep this in mind. They text you right aw...,sgcach,11302,508,https://www.reddit.com/r/dating_advice/comment...,uniqueruntimeerror,"Yeah, and even then they can turn 180 and pret...",144
...,...,...,...,...,...,...,...,...,...
55187,Guy told me I “should” check my cholesterol af...,So we had our first date yesterday. He sent me...,sf768x,1999,580,https://www.reddit.com/r/dating_advice/comment...,Fluffy_Telephone4258,Agreed!,2
55188,Guy told me I “should” check my cholesterol af...,So we had our first date yesterday. He sent me...,sf768x,1999,580,https://www.reddit.com/r/dating_advice/comment...,Anti-ThisBot-IB,https://i.imgur.com/KrwA19h.jpeg\n***\n ^(I am...,1
55189,Guy told me I “should” check my cholesterol af...,So we had our first date yesterday. He sent me...,sf768x,1999,580,https://www.reddit.com/r/dating_advice/comment...,Anti-ThisBot-IB,https://i.imgur.com/KrwA19h.jpeg\n***\n ^(I am...,1
55190,Guy told me I “should” check my cholesterol af...,So we had our first date yesterday. He sent me...,sf768x,1999,580,https://www.reddit.com/r/dating_advice/comment...,None,This.,-9


In [51]:
# Converting dataframes to csv
topPosts.to_csv("ScrapedTitles.csv")
top100.to_csv("Top100ScrapedTitles.csv")
allComments.to_csv("ScrapedComments.csv")
combinedScrape.to_csv("CombinedTop100Scrape.csv")